In [5]:
import numpy as np 
import pandas as pd 
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
conda install -c conda-forge folium


import json 


from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests 
from pandas.io.json import json_normalize 

import matplotlib.cm as cm
import matplotlib.colors as colors

from sklearn.cluster import KMeans

import folium

print('Libraries imported.')


SyntaxError: invalid syntax (<ipython-input-5-dd926a564a44>, line 5)

In [6]:

address = 'Rio de Janeiro, Rio de Janeiro'

geolocator = Nominatim(user_agent="rio_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Rio de Janeiro is {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Rio de Janeiro is -22.9110137, -43.2093727.


In [7]:
CLIENT_ID = 'XBVDVFI3BB3LBMSPWBVCYUDG0AGUZRQP1MFUXOS0SKACIAGK'
CLIENT_SECRET = 'HD34GAYDCO0GOUM1Z3NFCY4JH4FO0V5F4FMOLO4CEDSZO5AV' 
VERSION = '20180605' # Foursquare API version
LIMIT = 100
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: XBVDVFI3BB3LBMSPWBVCYUDG0AGUZRQP1MFUXOS0SKACIAGK
CLIENT_SECRET:HD34GAYDCO0GOUM1Z3NFCY4JH4FO0V5F4FMOLO4CEDSZO5AV


In [8]:
search_query = 'Restaurant'
radius = 50000
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=XBVDVFI3BB3LBMSPWBVCYUDG0AGUZRQP1MFUXOS0SKACIAGK&client_secret=HD34GAYDCO0GOUM1Z3NFCY4JH4FO0V5F4FMOLO4CEDSZO5AV&ll=-22.9110137,-43.2093727&v=20180605&query=Restaurant&radius=50000&limit=100'

In [9]:
results = requests.get(url).json()


In [10]:
venues = results['response']['venues']
dataframe = json_normalize(venues)
dataframe.head()

,categories,hasPerk,id,location.address,location.cc,location.city,location.country,location.crossStreet,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.postalCode,location.state,name,referralId,venuePage.id
0,"[{'id': '52e81612bcbc57f1066b79f1', 'name': 'B...",False,56e04e1d38fa3ae11155b863,"Rua Riachuelo, 242",BR,Rio de Janeiro,Brasil,NaN,2146,"[Rua Riachuelo, 242, Rio de Janeiro, RJ, Brasil]","[{'label': 'display', 'lat': -22.914733, 'lng'...",-22.914733,-43.188829,NaN,RJ,Riá Restaurant,v-1598979200,NaN
1,"[{'id': '4bf58dd8d48988d16b941735', 'name': 'B...",False,4ece6782cc219860f521dd5d,NaN,BR,Rio de Janeiro,Brasil,NaN,1828,"[Rio de Janeiro, RJ, Brasil]","[{'label': 'display', 'lat': -22.9097338088607...",-22.909734,-43.191598,NaN,RJ,Restaurante Pitada,v-1598979200,NaN
2,"[{'id': '4d4ae6fc7a7b7dea34424761', 'name': 'F...",False,56af89ea498e8dd1ce939e4a,"Rua Joaquim Palhares, 115",BR,Rio de Janeiro,Brasil,NaN,359,"[Rua Joaquim Palhares, 115, Rio de Janeiro, RJ...","[{'label': 'display', 'lat': -22.9139819676273...",-22.913982,-43.207982,20260-080,RJ,Restaurante Galeto Divino,v-1598979200,NaN
3,"[{'id': '4bf58dd8d48988d1c4941735', 'name': 'R...",False,50e0b634e4b0f94bd8a38f47,Navio Soberano,BR,Rio de Janeiro,Brasil,NaN,2849,"[Navio Soberano, Rio de Janeiro, RJ, Brasil]","[{'label': 'display', 'lat': -22.8928508209919...",-22.892851,-43.189795,NaN,RJ,Restaurant El Duero,v-1598979200,NaN
4,"[{'id': '4bf58dd8d48988d16b941735', 'name': 'B...",False,4d46d2732e326ea8e41afba6,"R. do Matoso, 30",BR,Rio de Janeiro,Brasil,Pç. da Bandeira,435,"[R. do Matoso, 30 (Pç. da Bandeira), Rio de Ja...","[{'label': 'display', 'lat': -22.9120948222039...",-22.912095,-43.213454,NaN,RJ,Restaurante Grão & Grão,v-1598979200,NaN


In [11]:
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

    dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered.head()

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,postalCode,state,id
0,Riá Restaurant,"[{'id': '52e81612bcbc57f1066b79f1', 'name': 'B...","Rua Riachuelo, 242",BR,Rio de Janeiro,Brasil,NaN,2146,"[Rua Riachuelo, 242, Rio de Janeiro, RJ, Brasil]","[{'label': 'display', 'lat': -22.914733, 'lng'...",-22.914733,-43.188829,NaN,RJ,56e04e1d38fa3ae11155b863
1,Restaurante Pitada,"[{'id': '4bf58dd8d48988d16b941735', 'name': 'B...",NaN,BR,Rio de Janeiro,Brasil,NaN,1828,"[Rio de Janeiro, RJ, Brasil]","[{'label': 'display', 'lat': -22.9097338088607...",-22.909734,-43.191598,NaN,RJ,4ece6782cc219860f521dd5d
2,Restaurante Galeto Divino,"[{'id': '4d4ae6fc7a7b7dea34424761', 'name': 'F...","Rua Joaquim Palhares, 115",BR,Rio de Janeiro,Brasil,NaN,359,"[Rua Joaquim Palhares, 115, Rio de Janeiro, RJ...","[{'label': 'display', 'lat': -22.9139819676273...",-22.913982,-43.207982,20260-080,RJ,56af89ea498e8dd1ce939e4a
3,Restaurant El Duero,"[{'id': '4bf58dd8d48988d1c4941735', 'name': 'R...",Navio Soberano,BR,Rio de Janeiro,Brasil,NaN,2849,"[Navio Soberano, Rio de Janeiro, RJ, Brasil]","[{'label': 'display', 'lat': -22.8928508209919...",-22.892851,-43.189795,NaN,RJ,50e0b634e4b0f94bd8a38f47
4,Restaurante Grão & Grão,"[{'id': '4bf58dd8d48988d16b941735', 'name': 'B...","R. do Matoso, 30",BR,Rio de Janeiro,Brasil,Pç. da Bandeira,435,"[R. do Matoso, 30 (Pç. da Bandeira), Rio de Ja...","[{'label': 'display', 'lat': -22.9120948222039...",-22.912095,-43.213454,NaN,RJ,4d46d2732e326ea8e41afba6


In [12]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=13) # generate map centred around the Conrad Hotel

folium.features.CircleMarker(
    [latitude, longitude],
    radius=10,
    color='red',
    popup='Conrad Hotel',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(venues_map)

for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map)

venues_map

NameError: name 'folium' is not defined